# NEB Tm Calculator - Phusion 

This notebook describe the replication of the NEB [Tm](https://tmcalculator.neb.com/#!/batch) settings 
using the Biopython [Bio.SeqUtils.MeltingTemp.Tm_NN](https://github.com/biopython/biopython/blob/d398dfacaae80ee2e438577172ff01f1eb1b7f7f/Bio/SeqUtils/MeltingTemp.py#L831) function.

Screenshots of NEB Tm calculator settings:

![](neb_phusion1.png)

![](neb_phusion2.png)

![](neb_phusion3.png)

200 primers were [analyzed](tmcalc_batch.txt) 


In [1]:
import csv
primerlist = []
with open('tmcalc_batch.txt', newline='') as csvfile:
    neb_file_reader = csv.reader(csvfile, delimiter='\t')
    for row in neb_file_reader:
        name, sequence, nebtm, *rest = row
        primerlist.append([name, sequence, nebtm])
print(len(primerlist))

200


In [2]:
from Bio.SeqUtils import MeltingTemp as mt
from prettytable import PrettyTable
t = PrettyTable(['Name', 'Sequence', "neb", "bp"])

for name, primer, nebtm in primerlist:
    # For Phusion, use this settings
    bptm = mt.Tm_NN(
            primer,
            nn_table=mt.DNA_NN3,
            Na=0,  # mM
            K=50,  # mM
            Tris=25,  # mM
            Mg=1.5,
            dnac1=500,  # nM
            dnac2=0,  # nM
            dNTPs=0.8,  # mM
            saltcorr=1,)    
    nebtm = int(nebtm)
    if nebtm != round(bptm):
        t.add_row([name, primer, nebtm, round(bptm, 4)])
print(t)

+------+--------------------+-----+---------+
| Name |      Sequence      | neb |    bp   |
+------+--------------------+-----+---------+
| p142 | AATACAGTCTGCTACTTA |  48 | 48.5204 |
| p154 | TTTATGTTACAACGAGAC |  49 | 48.4702 |
|  p1  | CCCTCTCTTTTGGATGTA |  52 | 52.5062 |
| p120 | ACTAGGGGTATTTCCAAC |  52 | 52.5352 |
| p136 | TGTACGCACGTATAGTAC |  53 | 52.4891 |
| p54  | TCAACTCTTGACAGCGAT |  55 | 55.5408 |
| p57  | TAAGGGGCCCGTCTTAAA |  57 | 57.5968 |
|  p7  | CTCACCTGGAGGTCCGGG |  63 | 63.5432 |
+------+--------------------+-----+---------+


8/200 primers analyzed had a Tm that rounded to a value 1°C from the neb result.
This means that the parameters used by neb are very similar to the one used here.